\pagebreak

## Traffic Signals.ipynb (Traffic Signals Analysis)

### Libraries and functions

In [ ]:
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import box
import matplotlib.pyplot as plt
import seaborn as sns

\pagebreak

### Reading and merging data

In [ ]:
#add traffic incidents
incidents_df = pd.read_csv('..\..\CSV_files\Traffic_Incidents.csv') #call csv
incidents_df['data']='TrafficIncidents' #create column to keep track of where this data came from when merging dataframes
#rename columns so they match other dataframes
incidents_df=incidents_df.rename(columns={'Latitude':'latitude','Longitude':'longitude','Count':'Incidents'})
#incidents_df.head() #<-- visual QC of dataframe

In [ ]:
#data given to 6 significant digets, so this doesnt actualy do much(but we kept the code, just incase we wanted to 
#experiment with radius)
incidents_df = incidents_df.round({'latitude':6, 'longitude':6})
#incidents_df #<-- visual QC of dataframe

In [ ]:
#add trafic signals
signals_df = pd.read_csv('..\..\CSV_files\Traffic_Signals.csv')
signals_df=signals_df.rename(columns={'Count':'Signal Count'})
signals_df['data']='TrafficSignals'
#signals_df #<-- visual QC of dataframe

In [ ]:
signals_df = signals_df.round({'latitude':6, 'longitude':6})

In [ ]:
#DID A LEFT JOIN, BECASUE MORE SIGNALS THAN ACCIDENTS DATA
accidents_vs_signals = pd.merge(left=signals_df, right=incidents_df, how='left', left_on=['latitude','longitude'], right_on=['latitude','longitude'])
#accidents_vs_signals #<-- visual QC of dataframe

\pagebreak

### Incidents and signal count vs. intersection type (Table)

In [ ]:
INT_TYPE_TABLE = accidents_vs_signals.groupby(['INT_TYPE']).sum()
INT_TYPE_TABLE = INT_TYPE_TABLE[INT_TYPE_TABLE.columns.difference(['latitude','longitude','ACCESSIBLE PEDESTRIAN SIGNAL'])]
INT_TYPE_TABLE['%Incidents to Signal Count'] = (INT_TYPE_TABLE['Incidents'] / INT_TYPE_TABLE['Signal Count'])*100
INT_TYPE_TABLE = INT_TYPE_TABLE.sort_values(['%Incidents to Signal Count'], ascending=False) #order table
INT_TYPE_TABLE #<---print table for analysis

\pagebreak

### Percentatge of incidents vs. signal type (Histogram)

In [ ]:
#create histogram of the above table
INT_TYPE_TABLE=INT_TYPE_TABLE.reset_index()
fig,ax = plt.subplots(figsize=(15,7))
sns.barplot(x = 'INT_TYPE', y = '%Incidents to Signal Count', data = INT_TYPE_TABLE)

plt.ylabel("% Incidents to Signal Count",fontsize=20)
plt.xlabel("Signal Type",fontsize=20)
plt.title("Percentatge of Incidents vs. Signal Type",fontsize=24)

\pagebreak

### Incident to signal count percentage in each city quadrant (Table)

In [ ]:
QUADRANT_TABLE = accidents_vs_signals.groupby(['QUADRANT_x']).sum()
QUADRANT_TABLE = QUADRANT_TABLE[QUADRANT_TABLE.columns.difference(['latitude','longitude','ACCESSIBLE PEDESTRIAN SIGNAL'])]
QUADRANT_TABLE['%Incidents to Signal Count'] = (QUADRANT_TABLE['Incidents'] / QUADRANT_TABLE['Signal Count'])*100
QUADRANT_TABLE = QUADRANT_TABLE.sort_values(['%Incidents to Signal Count'], ascending=False)
QUADRANT_TABLE=QUADRANT_TABLE.reset_index()
QUADRANT_TABLE #<-- show to see table

In [ ]:
#create histogram of the above table
fig,ax = plt.subplots(figsize=(15,7))
sns.barplot(x = 'QUADRANT_x', y = '%Incidents to Signal Count', data = QUADRANT_TABLE)

plt.ylabel("% Incidents to Signal Count",fontsize=20)
plt.xlabel("City Quadrant",fontsize=20)
plt.title("Incident to Signal Count Percentage in each City Quadrant",fontsize=24)

\pagebreak

### Pedestrian button at intersection correlation (Table)

In [ ]:
PEDBUTTON_TABLE = accidents_vs_signals.groupby(['PEDBUTTONS']).sum()
PEDBUTTON_TABLE = PEDBUTTON_TABLE[PEDBUTTON_TABLE.columns.difference(['latitude','longitude','ACCESSIBLE PEDESTRIAN SIGNAL'])]
PEDBUTTON_TABLE['%Incidents to Signal Count'] = (PEDBUTTON_TABLE['Incidents'] / PEDBUTTON_TABLE['Signal Count'])*100
PEDBUTTON_TABLE=PEDBUTTON_TABLE.reset_index()
PEDBUTTON_TABLE #<-- show to see table

\pagebreak

### Incident per signal count percentage for each quadrant in Calgary (BubblePlot)

In [ ]:
x=[0,1,0,0.5,1]
y=[0,1,1,0,0]
z=[1670,1020,394,4130,1357]
plt.scatter(x,y,s=z, c="red", alpha=0.5)

plt.ylabel("South           North",fontsize=20)
plt.xlabel("West                              East",fontsize=20)
plt.title("Incident per Signal Count Percentage for each Quadrant in Calgary",fontsize=24)

\pagebreak

### Incidents vs. signal count for each quadrant in Calgary (pointplot)

In [ ]:
#bins = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]

fig,ax = plt.subplots(figsize=(15,7))
sns.set(style="darkgrid")
g = sns.pointplot(x="Signal Count", y="Incidents", data=QUADRANT_TABLE, color="black")
g = sns.pointplot(x="Signal Count", y="Incidents", hue='QUADRANT_x', data=QUADRANT_TABLE, scale=2)

plt.ylabel("Incidents" ,fontsize=20)
plt.xlabel("Signal Count",fontsize=20)
plt.title("Incidents vs. Signal Count for each Quadrant in Calgary",fontsize=24)